In [ ]:
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from datetime import datetime
from matplotlib.pyplot import text
import pandas as pd
import cdflib
import numpy as np
import glob
import requests
import warnings
import matplotlib.ticker as mticker
fname = 'dmsp-f18_ssj_precipitating-electrons-ions_20120101_v1.1.1.cdf'
cdf_file = cdflib.CDF(fname)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
data_variables=cdf_file.cdf_info()['zVariables']
data_vars=[]
df=pd.DataFrame()
for variable in data_variables:
    if cdf_file.varget(variable).ndim !=1:
        Data=cdf_file.varget(variable)
        _,c=Data.shape
        for i in range(c):
            df[variable +str(i)] = pd.DataFrame(Data[:,i])
            data_vars.append(variable +str(i))  
    else:
        df[variable]=pd.DataFrame(cdf_file.varget(variable))
        data_vars.append(variable)

In [ ]:
## collecting the right columns for averaging here only variable anmes start with ELE and ION
ix=[item.startswith('ELE') for item in data_vars]
a=list(np.array(data_vars)[ix])
ix=[item.startswith('ION') for item in data_vars]
b=list(np.array(data_vars)[ix])
aa=a+b

In [ ]:
ix = df['SC_AACGM_LAT']>40 # Northern hemsiphere
df_NH=df[ix]
df_NH=df_NH.round()
 
f=df_NH.groupby(["SC_AACGM_LTIME", "SC_AACGM_LAT"])[aa].apply(lambda g: g.mean(skipna=True))
f=f.reset_index()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Define the grid parameters


# Generate a random dataset to grid
lon = df['SC_AACGM_LON'].values
lat = f['SC_AACGM_LAT'].values
data = df["ELE_DIFF_ENERGY_FLUX8"]

lon_range = (lon.min(), lon.max())  # Longitude range
lat_range = (lat.min(), lat.max())  # Latitude range
grid_size = 0.1  # Grid size in degrees

# Define the grid arrays
lon_grid = np.arange(lon_range[0], lon_range[1] + grid_size, grid_size)
lat_grid = np.arange(lat_range[0], lat_range[1] + grid_size, grid_size)
lon_grid, lat_grid = np.meshgrid(lon_grid, lat_grid)


# Grid the data using bilinear interpolation
from scipy.interpolate import griddata
data_grid = griddata((lon, lat), data, (lon_grid, lat_grid), method='linear')

proj = ccrs.Stereographic(central_longitude=0, central_latitude=90)

# Convert the gridded data to polar coordinates
theta = np.radians(lon_grid)
r = 90 - lat_grid
data_grid_polar = data_grid

# Plot the gridded data in a polar plot
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1, projection='polar')
ax.set_theta_zero_location('N')
ax.set_theta_direction(-1)
ax.set_rlim([0, 50])
cax = ax.contourf(theta, r, data_grid_polar, cmap='jet')
cb = fig.colorbar(cax, orientation='horizontal', pad=0.05)
cb.set_label('Data')
plt.show()

In [ ]:
df

In [ ]:
## dmsp_grid
lati = np.arange(f["SC_AACGM_LAT"].min(), f["SC_AACGM_LAT"].max()+1)
tt = np.arange(f["SC_AACGM_LTIME"].min(), f["SC_AACGM_LTIME"].max()+1)
x, y = np.meshgrid(tt, lati)

mlt = f["SC_AACGM_LTIME"]
latitude = f["SC_AACGM_LAT"]
ch=8
ePrep = f["ELE_DIFF_ENERGY_FLUX"+str(ch)]
iPrep = f["ION_DIFF_ENERGY_FLUX"+str(ch)]
egrided_data = np.nan*(np.ones(x.shape))
igrided_data = np.nan*(np.ones(x.shape))

for i in range(len(x[0, :])):
    ii = np.where(np.in1d(mlt.values, x[0, i]))
    # np.array(latitude[ii[0]])
    mm = np.where(np.in1d(y[:, 0], np.array(latitude[ii[0]])))
    egrided_data[mm, i] = ePrep[ii[0]]
    igrided_data[mm, i] = iPrep[ii[0]]
    
from dmsp_reader import dmsp_polar_plot
a=dmsp_polar_plot(x,y,egrided_data, igrided_data, ch=ch, savefig=False)